In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import math

In [117]:
    transforms = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.MNIST(
        './data',
        train=True,
        download=True,
        transform=transforms)

    test_dataset = datasets.MNIST(
        './data',
        train=False,
        download=True,
        transform=transforms
    )

In [195]:
    BATCH_SIZE = 128    
    N_EPOCHS = 50       
    INPUT_DIM = 784    
    HIDDEN_DIM = 512   
    LATENT_DIM = 2  
    lr = 0.001           

In [196]:
    train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [197]:
 class Encoder(nn.Module):
        
        def __init__(self, input_dim, hidden_dim, z_dim):
           
            super().__init__()

            self.linear = nn.Linear(input_dim, hidden_dim)
            self.mu = nn.Linear(hidden_dim, z_dim)
            self.var = nn.Linear(hidden_dim, z_dim)

        def forward(self, x):
            # x is of shape [batch_size, input_dim]

            hidden = F.relu(self.linear(x))
            # hidden is of shape [batch_size, hidden_dim]
            z_mu = self.mu(hidden)
            # z_mu is of shape [batch_size, latent_dim]
            z_var = self.var(hidden)
            # z_var is of shape [batch_size, latent_dim]

            return z_mu, z_var

In [198]:
  class Decoder(nn.Module):
        
        def __init__(self, z_dim, hidden_dim, output_dim):
            
            super().__init__()

            self.linear = nn.Linear(z_dim, hidden_dim)
            self.out = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            # x is of shape [batch_size, latent_dim]

            hidden = F.relu(self.linear(x))
            # hidden is of shape [batch_size, hidden_dim]

            predicted = torch.sigmoid(self.out(hidden))
            # predicted is of shape [batch_size, output_dim]

            return predicted


In [199]:
  class VAE(nn.Module):
        
        def __init__(self, enc, dec):
            super().__init__()

            self.enc = enc
            self.dec = dec

        def forward(self, x):
            # encode
            z_mu, z_var = self.enc(x)

            std = torch.exp(z_var / 2)
            eps = torch.randn_like(std)
            x_sample = eps.mul(std).add_(z_mu)

            # decode
            predicted = self.dec(x_sample)
            return predicted, z_mu, z_var

In [200]:
    # encoder
    encoder = Encoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM)

    # decoder
    decoder = Decoder(LATENT_DIM, HIDDEN_DIM, INPUT_DIM)

    # vae
    model = VAE(encoder, decoder).to(device)

    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

In [201]:
 def train():
        # set the train mode
        model.train()

        # loss of the epoch
        train_loss = 0

        for i, (x, _) in enumerate(train_iterator):
            # reshape the data into [batch_size, 784]
            x = x.view(-1, 28 * 28)
            x = x.to(device)
            
            # update the gradients to zero
            optimizer.zero_grad()

            # forward pass
            x_sample, z_mu, z_var = model(x)

            # reconstruction loss
            recon_loss = F.binary_cross_entropy(x_sample, x, size_average=False)

            # kl divergence loss
            kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1.0 - z_var)

            # total loss
            loss = recon_loss + kl_loss

            # backward pass
            loss.backward()
            train_loss += loss.item()
            
            # update the weights
            optimizer.step()

        return train_loss

In [202]:
  def test():
        # set the evaluation mode
        model.eval()

        # test loss for the data
        test_loss = 0

        # we don't need to track the gradients, since we are not updating the parameters during evaluation / testing
        with torch.no_grad():
            for i, (x, _) in enumerate(test_iterator):
                # reshape the data
                x = x.view(-1, 28 * 28)
                x = x.to(device)

                # forward pass
                x_sample, z_mu, z_var = model(x)

                # reconstruction loss
                recon_loss = F.binary_cross_entropy(x_sample, x, size_average=False)
                
                # kl divergence loss
                kl_loss = 0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1.0 - z_var)
                
                # total loss
                loss = recon_loss + kl_loss
                test_loss += loss.item()

        return test_loss


In [ ]:
    best_test_loss = float('inf')

    for e in range(N_EPOCHS):

        train_loss = train()
        test_loss = test()

        train_loss /= len(train_dataset)
        test_loss /= len(test_dataset)

        print(f'Epoch {e}, Train Loss: {train_loss:.2f}, Test Loss: {test_loss:.2f}')

In [ ]:
    # sample and generate a image
    z = torch.randn(1, LATENT_DIM).to(device)

    # run only the decoder
    reconstructed_img = model.dec(z)
    img = reconstructed_img.view(28, 28).data

    print(z.shape)
    print(img.shape)

    plt.imshow(img, cmap='gray')